In [1]:
import pandas as pd
import numpy as np
import json
import re # Regular expresions

from langdetect import detect

# Read data

In [2]:
# Open and decode the files
whayner_file = open('../scraper-data-whayner/data.json', encoding="utf8")
whayner_data = json.load(whayner_file)
whayner_dataframe = pd.DataFrame(whayner_data['videos'])

silvia_file = open('../scraper-data-silvia/data.json', encoding="utf8")
silvia_data = json.load(silvia_file)
silvia_dataframe = pd.DataFrame(silvia_data['videos'])

estefany_file = open('../scraper-data-estefany/data.json', encoding="utf8")
estefany_data = json.load(estefany_file)
estefany_dataframe = pd.DataFrame(estefany_data['videos'])

pedro_gomez_file = open('../scraper-data-pedrofelipe/data.json', encoding="utf8")
pedro_gomez_data = json.load(pedro_gomez_file)
pedro_gomez_dataframe = pd.DataFrame(pedro_gomez_data['videos'])

andres_file = open('../scraper-data-andres/data.json', encoding="utf8")
andres_data = json.load(andres_file)
andres_dataframe = pd.DataFrame(andres_data['videos'])

pedro_chaparro_file = open('../scraper-data-pedroandres/data.json', encoding="utf8")
pedro_chaparro_data = json.load(pedro_chaparro_file)
pedro_chaparro_dataframe = pd.DataFrame(pedro_chaparro_data['videos'])

# Concatenate in a single dataframe
complete_dataframe = pd.concat([whayner_dataframe, 
                               silvia_dataframe, 
                               estefany_dataframe, 
                               pedro_gomez_dataframe, 
                               andres_dataframe,
                               pedro_chaparro_dataframe])

# Reset index
complete_dataframe = complete_dataframe.reset_index(drop=True)


# print(complete_dataframe.shape)

In [42]:
# complete_dataframe.head()

In [43]:
# complete_dataframe.tail()

In [44]:
complete_dataframe.to_json('./initial.json', force_ascii=False, orient='table', index=False)

# Convert to lowercase letters

In [3]:
for dataframe_index in complete_dataframe.index: 
    title = complete_dataframe['title'][dataframe_index]
    description = complete_dataframe['description'][dataframe_index]
    tags = complete_dataframe['tags'][dataframe_index]
    
    complete_dataframe['title'][dataframe_index]= title.lower()
    complete_dataframe['description'][dataframe_index] = description.lower()
    complete_dataframe['tags'][dataframe_index] = tags.lower()

# Remove links 

In [4]:
for dataframe_index in complete_dataframe.index: 
    title = complete_dataframe['title'][dataframe_index]
    description = complete_dataframe['description'][dataframe_index]
    # Replace strings that starting with http with ''
    complete_dataframe['title'][dataframe_index] =  re.sub(r'http\S+', '', title, flags=re.MULTILINE)
    complete_dataframe['description'][dataframe_index] = re.sub(r'http\S+', '', description, flags=re.MULTILINE) 

# Remove unwanted characters (All that is not alfanum)

In [6]:
for dataframe_index in complete_dataframe.index: 
    title = complete_dataframe['title'][dataframe_index]
    description = complete_dataframe['description'][dataframe_index]
    tags = complete_dataframe['tags'][dataframe_index]

    # Remove not alfanumeric chars (with exceptions)
    new_title = re.sub(r'[^a-zA-Z0-9ñÑáéíóúÁÉÍÓÚ ]+', '', title)
    new_description = re.sub(r'[^a-zA-Z0-9ñÑáéíóúÁÉÍÓÚ ]+', '', description)
    new_tags = re.sub(r'[^a-zA-Z0-9ñÑáéíóúÁÉÍÓÚ ]+', '', tags)
    
    # Remove redundant spaces
    new_title = re.sub(' +', ' ', new_title).strip()
    new_description = re.sub(' +', ' ', new_description).strip()
    new_tags = re.sub(' +', ' ', new_tags).strip()

    # Replace
    complete_dataframe['title'][dataframe_index] = new_title
    complete_dataframe['description'][dataframe_index] = new_description
    complete_dataframe['tags'][dataframe_index] = new_tags

# Remove entries whose language is not english or spanish

In [7]:
accepted_languages = ['es', 'en', 'it']
removed = []

# Create a copy of original dataframe
df = complete_dataframe.copy()

for dataframe_index in complete_dataframe.index: 
    video_texts = complete_dataframe['description'][dataframe_index] + complete_dataframe['title'][dataframe_index] + complete_dataframe['tags'][dataframe_index]
    language = detect(video_texts)
    
    if(language not in accepted_languages):
        # print(language)
        # print(complete_dataframe['url'][dataframe_index])
        removed.append({'lang': language, 'url': complete_dataframe['url'][dataframe_index]})
        df = df.drop(dataframe_index)

print(removed)

[{'lang': 'af', 'url': 'https://www.youtube.com/watch?v=mnThZdxCTrw'}, {'lang': 'nl', 'url': 'https://www.youtube.com/watch?v=xshpILh3cLs'}, {'lang': 'da', 'url': 'https://www.youtube.com/watch?v=mrHQRJL0W84'}, {'lang': 'fr', 'url': 'https://www.youtube.com/watch?v=2WPTjHGNOpg'}, {'lang': 'da', 'url': 'https://www.youtube.com/watch?v=H_Zt3Vy6Suc'}, {'lang': 'da', 'url': 'https://www.youtube.com/watch?v=7R1VUePOqxE'}, {'lang': 'ca', 'url': 'https://www.youtube.com/shorts/8_pH8CnPAuw'}, {'lang': 'da', 'url': 'https://www.youtube.com/watch?v=ILPex8f-CYM'}, {'lang': 'ca', 'url': 'https://www.youtube.com/watch?v=jiIB_QEBMfc'}, {'lang': 'da', 'url': 'https://www.youtube.com/watch?v=BWRNNXa-S3Y'}, {'lang': 'fr', 'url': 'https://www.youtube.com/watch?v=0b-keN3GEO4'}, {'lang': 'fr', 'url': 'https://www.youtube.com/watch?v=SBXoQH1ufmQ'}, {'lang': 'fr', 'url': 'https://www.youtube.com/watch?v=Bw644d2L8wo'}, {'lang': 'ca', 'url': 'https://www.youtube.com/watch?v=-jMLpq4agH4'}, {'lang': 'fr', 'url'

In [8]:
complete_dataframe.shape

(12849, 5)

In [49]:
df.shape

(12619, 5)

# Save as json

In [50]:
df.to_json('./data.json', force_ascii=False, orient='table', index=False)